In [ ]:
import nest_asyncio
nest_asyncio.apply()

from hddCRP.simulations import simulate_sessions
from hddCRP.modelBuilder import cdCRP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import arviz as az

import os
import sys

In [ ]:



args = [0, 1, 1]
# if __name__ == "__main__":
#     for ii, arg in enumerate(sys.argv):
#         if(ii < len(args)):
#             args[ii] = arg

simulation_id = args[0];
min_blocks = args[1];
max_blocks = args[2];

run_range = range(0, 20)
results_directory = "Results/Simulations/"
OVERWRITE = False;


if(simulation_id == 0):
    alpha = 2
    different_context_weights = [0.8, 0.8];
    within_session_timescales  = {"A" : 20}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.75

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 * n_blocks];
    num_subjects = lambda n_blocks : 1;
    num_responses = 3
elif(simulation_id == 1):
    alpha = 8
    different_context_weights = [0.5, 0.5];
    within_session_timescales  = {"A" : 50}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.25

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 * n_blocks];
    num_subjects = lambda n_blocks : 1;
    num_responses = 3
elif(simulation_id == 2):
    alpha = 2
    different_context_weights = [0.8, 0.8];
    within_session_timescales  = {"A" : 20}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.75

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 ];
    num_subjects = lambda n_blocks : n_blocks;
    num_responses = 3
elif(simulation_id == 3):
    alpha = 8
    different_context_weights = [0.5, 0.5];
    within_session_timescales  = {"A" : 50}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.25

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 ];
    num_subjects = lambda n_blocks :  n_blocks;
    num_responses = 3
elif(simulation_id == 4):
    alpha = 2
    different_context_weights = [0.8, 0.8];
    within_session_timescales  = {"A" : 50}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.75

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 ];
    num_subjects = lambda n_blocks : n_blocks;
    num_responses = 3
else:
    raise NotImplementedError("No sim found")

repeat_bias_1_back = 1;
block_range = range(min_blocks, max_blocks+1)

if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)



In [ ]:



for block_idx in block_range:
    print(f"BLOCK {block_idx}")
    fit_file = f"{results_directory}/sim_{simulation_id}_block_{block_idx}.pkl"
    fit_summary_file = f"{results_directory}/sim_summary_{simulation_id}_block_{block_idx}.pkl"
    if(((not os.path.isfile(fit_file)) or (not os.path.isfile(fit_summary_file))) or OVERWRITE):
        simulation_fits = pd.DataFrame()
        simulation_fit_metrics = pd.DataFrame()
        for run_idx in run_range:
            print(f"BLOCK {block_idx} - RUN {run_idx}")
            sim_seed  = (simulation_id+1) * 10000 + run_idx*100
            stan_seed = (simulation_id+1) * 10000 + run_idx*100 + block_idx
            sim_rng = np.random.Generator(np.random.MT19937(sim_seed))
            
            seqs = [];
            subject_labels = [];
            session_labels_all = [];
            for jj in range(num_subjects(block_idx)):
                seqs_c = simulate_sessions(session_lengths=session_lengths(block_idx), session_labels=session_labels(block_idx), num_responses=num_responses, 
                                        alpha=alpha,
                                        different_context_weights=different_context_weights,
                                        within_session_timescales=within_session_timescales, between_session_timescales=between_session_timescales,
                                        repeat_bias_1_back=repeat_bias_1_back, rng=sim_rng)
                subject_labels += [jj] * len(seqs_c)
                session_labels_all += session_labels(block_idx)
                seqs += seqs_c;

            model = cdCRP(seqs, subject_labels=subject_labels, session_labels=session_labels_all);
            model.same_nback_depth = 0;

            model.build(random_seed=stan_seed);
            model.fit_model()

            map_fit = model.get_map()
            fit_df = model.fit.to_frame()
            fit_df["block"] = block_idx
            fit_df["run"]   = run_idx
            fit_df["simulation"]   = simulation_id
            summary_df = model.fit_summary()
            summary_df["block"] = block_idx
            summary_df["run"]   = run_idx
            summary_df["MAP"] = pd.Series(map_fit)
            summary_df["simulation"]   = simulation_id

            simulation_fit_metrics = pd.concat([simulation_fit_metrics,summary_df], copy=False)
            simulation_fits = pd.concat([simulation_fits,fit_df], copy=False)

        simulation_fits.to_pickle(fit_file)
        simulation_fit_metrics.to_pickle(fit_summary_file)
    else:
        print("Fit files found: not overriding")

In [24]:
simulation_fits
# observations are independent given a latent variable: a random measure  DP(alpha G)

In [25]:
model.to_dict()["subject_start_idx"]